<a href="https://colab.research.google.com/github/GeomaticsCaminosUPM/footprint_attributes/blob/main/examples/example_iiregularity_norms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Footprint Attributes package example

Install libraries if using colab

In [13]:
!pip install geopandas
!pip install git+https://github.com/GeomaticsCaminosUPM/SeismicBuildingExposure.git
!pip install folium matplotlib mapclassify

In [14]:
import SeismicBuildingExposure.irregularity
import geopandas as gpd

Documentation of every function can be found using `help()` or under https://github.com/GeomaticsCaminosUPM/SeismicBuildingExposure

In [15]:
help(SeismicBuildingExposure.irregularity.mexico_NTC_df)

Help on function mexico_NTC_df in module SeismicBuildingExposure.irregularity:

mexico_NTC_df(geoms: geopandas.geodataframe.GeoDataFrame, max_slenderness=4) -> pandas.core.frame.DataFrame



## 1. Load data

Load the footprints geodataframe in `.shp` `.gpkg` `.geojson` format using `geopandas.read_file()`

Download an example footprints file if needed

In [16]:
!curl -L -o footprints.gpkg https://github.com/GeomaticsCaminosUPM/SeismicBuildingExposure/raw/refs/heads/main/examples/footprints.gpkg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

100  364k  100  364k    0     0   425k      0 --:--:-- --:--:-- --:--:--  425k


Load the footprints geodataframe in `.shp` `.gpkg` `.geojson` format using `geopandas.read_file()`

In [17]:
footprints_gdf = gpd.read_file('footprints.gpkg')

- All geometries should be single part and polygons.

    - Multi part geometries (MultiPolygon) would mean two buildings are contained in the same footprint geometry which should not happen.

    - Footprints must be polygons, not linestrings.

Check if all geometries are `Polygon`

In [18]:
if any(footprints_gdf.geometry.type == 'MultiPolygon'):
    print("There are multiplart geometries.")

if any(footprints_gdf.geometry.type.str.contains('Polygon') == False):
    print("Some geometries are not Polygon")

There are multiplart geometries.


Explode multipart geometries into single parts if needed


Note: The row number will be reset. Maybe you do not know how to match the footprints with your data. An id column is a good idea to solve this issue.

In [19]:
footprints_gdf = footprints_gdf.explode().reset_index() # The new index column is the row number in the origninal dataframe
footprints_gdf

,index,gid,geometry
0,0,1218.0,"POLYGON ((488600.239 1097577.417, 488591.045 1..."
1,1,1219.0,"POLYGON ((488562.593 1097585.096, 488567.403 1..."
2,2,1220.0,"POLYGON ((488601.331 1097581.438, 488599.351 1..."
3,3,1221.0,"POLYGON ((488517.642 1097593.605, 488517.592 1..."
4,4,1222.0,"POLYGON ((488492.697 1097591.081, 488492.546 1..."
...,...,...,...
939,943,1273.0,"POLYGON ((488958.217 1097631.443, 488961.413 1..."
940,944,1333.0,"POLYGON ((489119.978 1097677.125, 489130.663 1..."
941,945,3632.0,"POLYGON ((489024.946 1097763.628, 489024.157 1..."
942,946,1505.0,"POLYGON ((488924.894 1097824.766, 488928.927 1..."


## 2. Eurocode 8

Irregularity is measured following the [Eurocode 8 standard](https://www.phd.eng.br/wp-content/uploads/2015/02/en.1998.1.2004.pdf).

The calculated parameters are:

- **Excentricity Ratio**:  
  $
  \text{excentricity\_ratio} = \frac{\text{excentricity}}{\text{torsional radius}}
  $  
  This considers the worst-case value across all possible directions.

- **Radius Ratio**:  
  $
  \text{radius\_ratio} = \frac{\text{torsional radius}}{\text{radius of gyration}}
  $


- **Slenderness**:  
  Typically calculated as $ \frac{L_1}{L_2} $, where $ L_1 $ and $ L_2 $ are the sides of the footprint.  
  Since polygonal shapes may not clearly resemble rectangles, we use:  
  $
  \sqrt{\frac{I_1}{I_2}}
  $  
  where $ I_1 $ and $ I_2 $ are the principal values of the inertia tensor.

- **Compactness**:  
  $
  \text{compactness} = \frac{\text{area of polygon (with all holes filled)}}{\text{area of convex hull}}
  $

The function `eurocode_8_df` returns the **weakest direction** as an angle (in degrees) with respect to the **north (in UTM coordinates)**.

### Definitions:

- **Excentricity**: Distance between the center of mass (centroid of the polygon) and the center of stiffness (centroid of the boundary).
- **Torsional Radius**:  
  $
  \sqrt{\frac{I_t}{I_j}}
  $  
  where:
  - $ I_t $: Inertia in the Z-direction through the center of stiffness.  
  - $ I_j $: Inertia through the center of mass along the axis perpendicular to the calculation direction.

- **Radius of Gyration**:  
  $
  \sqrt{\frac{I_0}{\text{area}}}
  $  
  where:
  - $ I_0 $: Inertia in the Z-direction through the center of mass.

### Eurocode 8 Parameter Limits:

| Parameter             | Limit        |
|----------------------|--------------|
| Excentricity Ratio    | < 0.3        |
| Radius Ratio          | < 1.0        |
| Slenderness           | < 4.0        |
| Compactness           | > 0.95       |



In [20]:
result = SeismicBuildingExposure.irregularity.eurocode_8_df(footprints_gdf)
result

,excentricity_ratio,radius_ratio,slenderness,compactness,angle_excentricity,angle_slenderness
0,0.245076,0.349005,1.232601,1.000000,-82.598668,-75.086654
1,0.008968,0.163555,3.192047,0.999683,-16.032497,-86.104788
2,0.044199,0.279863,2.961852,0.999880,45.056284,-65.468554
3,0.014583,0.232617,3.338191,0.997752,-52.235437,-4.542072
4,0.255400,0.223321,2.101109,0.859100,88.389159,-10.539551
...,...,...,...,...,...,...
939,0.622027,0.214324,1.100164,0.909876,-36.951387,-6.732787
940,0.011061,0.297200,1.158917,1.000000,-56.189202,-12.724535
941,0.185631,0.340129,1.739688,0.875405,45.307141,-76.976761
942,0.158282,0.439820,1.350890,0.893873,-13.396754,-62.699325


Save relevant columns of the result to the `footprints_gdf` variable if needed

In [21]:
footprints_gdf['excentricity_ratio_EC8'] = result['excentricity_ratio']
footprints_gdf['radius_ratio_EC8'] = result['radius_ratio']
footprints_gdf['slenderness_EC8'] = result['slenderness']
footprints_gdf['compactness_EC8'] = result['compactness']

Plot a map of building footprints that meet the Eurocode 8 excentricity ratio criterion.

In [22]:
m=footprints_gdf.loc[footprints_gdf['excentricity_ratio_EC8'] <= 0.3].explore(color='green')
m=footprints_gdf.loc[footprints_gdf['excentricity_ratio_EC8'] > 0.3].explore(m=m,color='red')
m

## 3. Costa Rica Código Sísmico Norm

Irregularity is measured following the [Seismic Code of Costa Rica](https://www.codigosismico.or.cr/).

The calculated parameter is:

- **Excentricity Ratio**:  
  $
  \text{excentricity\_ratio} = \frac{\text{excentricity}}{\text{dimension}}
  $  
  considering the **weakest possible direction**.

The function `codigo_sismico_costa_rica_df` returns the **weakest direction** as an angle (in degrees) with respect to the **north (in UTM coordinates)**.

### Definitions:

- **Excentricity**:  
  The distance between the **center of mass** (centroid of the polygon) and the **center of stiffness** (centroid of the boundary).

- **Dimension**:  
  The length of the shape in the considered direction. For rectangles, this is straightforward, but for an arbitrary polygon it is computed as:  
  $
  \text{dimension} = \sqrt{\text{area} \cdot \sqrt{\frac{I_i}{I_j}}}
  $  
  where:
  - $ I_i $: Inertia in the considered direction (through the center of mass).
  - $ I_j $: Inertia in the perpendicular direction (also through the center of mass).

### Código Sísmico de Costa Rica – Parameter Limits:

| Parameter            | Limit                        | Irregularity Level |
|---------------------|------------------------------|--------------------|
| Excentricity Ratio   | $< 0.05$                      | Regular             |
| Excentricity Ratio   | $0.05 < \frac{e}{d} < 0.25$   | Moderate            |
| Excentricity Ratio   | $> 0.25$                      | High                |



In [23]:
result = SeismicBuildingExposure.irregularity.codigo_sismico_costa_rica_df(footprints_gdf)
result

,excentricity_ratio,angle
0,0.032608,-81.520791
1,0.001232,-3.748371
2,0.009637,33.439118
3,0.001942,-54.645441
4,0.039722,84.707905
...,...,...
939,0.053691,-36.706984
940,0.001342,-56.123849
941,0.032035,41.327770
942,0.030572,-12.771424


Save relevant variables to `footprints_gdf`

In [24]:
footprints_gdf['excentricity_ratio_CR'] = result['excentricity_ratio']

Plot a map of building footprints that meet the Costa Rica excentricity ratio criterion.

In [25]:
m=footprints_gdf.loc[footprints_gdf['excentricity_ratio_CR'] <= 0.05].explore(color='green')
m=footprints_gdf.loc[(footprints_gdf['excentricity_ratio_CR'] > 0.05) & (footprints_gdf['excentricity_ratio_CR'] < 0.25)].explore(m=m,color='yellow')
m=footprints_gdf.loc[footprints_gdf['excentricity_ratio_CR'] > 0.25].explore(m=m,color='red')
m

C:\Users\Miguel\AppData\Local\Temp\ipykernel_13408\55075204.py:3: UserWarning: The GeoSeries you are attempting to plot is composed of empty geometries. Nothing has been displayed.
  m=footprints_gdf.loc[footprints_gdf['excentricity_ratio_CR'] > 0.25].explore(m=m,color='red')


## 4. Mexico NTC norm

Irregularity is measured following the [Mexico NTC norm]().

The calculated parameters are:

- **Setback Ratio**:  
  $
  \text{setback\_ratio} = \frac{\text{length of setback}}{\text{length of side}}
  $  
  considering the **worst of the two directions** and the **worst of all setbacks**.

- **Holes Ratio**:  
  $
  \text{holes\_ratio} = \frac{\text{width of hole}}{\text{length of side}}
  $  
  considering the **worst of the two directions** and the **worst of all holes**.

### Definitions:

- **Length of Side**:  
  The footprint is circumscribed in the **smallest possible rectangle**, with sides aligned to the **principal axes of the inertia tensor** of the footprint.  
  The *length of side* refers to the side of this rectangle along either of the principal directions.

  For the **holes ratio**, we consider the **principal axes of each hole shape**, and measure the side length as a line passing through the **center of mass of the hole** in each principal direction.

- **Length of Setback**:  
  Setbacks are defined as the **polygons formed by the difference between the convex hull and the footprint (with holes filled)**.  
  These setback polygons are also circumscribed in a rectangle whose sides are aligned with the **principal directions of the inertia tensor of the footprint**.  
  The *length of setback* is the side of this rectangle in one of the two principal directions.  
  In the **setback ratio**, both the *length of setback* and the *length of side* must be taken in the **same direction** (parallel).

- **Width of Hole**:  
  Each hole is circumscribed in a rectangle, with sides aligned to the **principal directions of the hole’s inertia tensor**.  
  The *width of the hole* is the length of this rectangle in each principal direction.

- **Max Slenderness**: In the context of the **setback ratio**, very thin irregularities caused by concave angles close to 180º comming from imperfections in the footprint,  can lead to a circumscribed rectangle with a disproportionately large side, even though such features may not represent a true setback. **Max slenderness** is the maximum slenderness of **setback circunscribed rectangles**. 

### Mexico NTC – Parameter Limits:

| Parameter        | Limit |
|------------------|--------|
| Setback Ratio     | $< 0.4$ |
| Hole Ratio        | $< 0.4$ |


In [26]:
result = SeismicBuildingExposure.irregularity.mexico_NTC_df(footprints_gdf, max_slenderness=4)
result

,setback_ratio,hole_ratio
0,0.000000,0.0
1,0.000000,0.0
2,0.000000,0.0
3,0.000000,0.0
4,0.691307,0.0
...,...,...
939,0.394207,0.0
940,0.000000,0.0
941,0.520152,0.0
942,0.476095,0.0


Save relevant variables

In [27]:
footprints_gdf['setback_ratio_NTC'] = result['setback_ratio']
footprints_gdf['hole_ratio_NTC'] = result['hole_ratio']

Plot a map of building footprints that meet the Mexico NTC setback ratio criterion.

In [28]:
m=footprints_gdf.loc[footprints_gdf['setback_ratio_NTC'] <= 0.4].explore(color='green')
m=footprints_gdf.loc[footprints_gdf['setback_ratio_NTC'] > 0.4].explore(m=m,color='red')
m

## 5. Save results

In [ ]:
footprints_gdf.to_file('path/to/save/file.gpkg')